In [3]:
import pandas as pa
import numpy as np
import os
print("import done")

import done


In [4]:
#data  = pa.read_csv("COVID 19 Containment measures data.csv")
os.getcwd()
os.listdir()

['.git',
 '.ipynb_checkpoints',
 'COVID 19 Containment measures data.csv',
 'data',
 'Pandemiculator.ipynb',
 'README.md']

In [54]:
row_data  = pa.read_csv(r"C:\Users\praveen barasia\Downloads\tarang\GitHub\Pandemiculator_COVID19\data\IndianData.csv",sep=",",encoding='cp1252')
data= pa.DataFrame(row_data)
Column_Name=data.columns
Column_Name
Indian_data=data[['Patient Number', 'Date Announced',
        'Age Bracket', 'Gender','Detected State', 'Current Status',
       'Type of transmission']]
Indian_data

,Patient Number,Date Announced,Age Bracket,Gender,Detected State,Current Status,Type of transmission
0,1,30/01/2020,20,F,Kerala,Recovered,Imported
1,2,02/02/2020,NaN,NaN,Kerala,Recovered,Imported
2,3,03/02/2020,NaN,NaN,Kerala,Recovered,Imported
3,4,02/03/2020,45,M,Delhi,Recovered,Imported
4,5,02/03/2020,24,M,Telangana,Recovered,Imported
...,...,...,...,...,...,...,...
6628,6629,NaN,NaN,NaN,NaN,NaN,NaN
6629,6630,NaN,NaN,NaN,NaN,NaN,NaN
6630,6631,NaN,NaN,NaN,NaN,NaN,NaN
6631,6632,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
#Indian_df.index
#Indian_df.describe

